In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import pickle
import numpy as np

In [ ]:
!pip install matplotlib
!pip install numpy

In [ ]:
with open('sensitive_analysis_44.pkl','r') as inputs:
    data = pickle.load(inputs)
with open('sensitive_analysis_base.pkl','r') as inputsbase:
    data_base = pickle.load(inputsbase)

In [ ]:
def get_price_stat_ave(data,decision_times):
    mitigation = []
    utility = []
    price = []
    from tree import TreeModel 
    for i in range(len(data)):
        mitigation.append(data[i][2])
        utility.append(data[i][3])
        price.append(data[i][4])
    
    t = TreeModel(decision_times)
    nodes_index = []
    for i in range(t.num_periods):
        nodes_index.append(t.get_nodes_in_period(i))
    price_stat_ave = np.zeros((len(data), t.num_periods))
    for i in range(len(data)):
        for j in range(t.num_periods):
            price_stat_ave[i,j] = np.average(data[i][4][nodes_index[j][0]:nodes_index[j][1]+1])
    return price_stat_ave
def get_90_quantile(price_stat_ave):
    list0 = price_stat_ave[:,0]
    list1 = price_stat_ave[:,1]
    list2 = price_stat_ave[:,2]
    list3 = price_stat_ave[:,3]
    list4 = price_stat_ave[:,4]
    list5 = price_stat_ave[:,5]
    list_ = [list0,list1,list2,list3,list4,list5]
    for i in range(len(list_)):
        list_[i]=np.sort(list_[i])[2:42]
    list_min = []
    list_max = []
    for i in list_:
        list_min.append(i[0])
        list_max.append(i[-1])   
    return list_min,list_max

In [ ]:
decision_times=[0, 15, 45, 85, 185, 285, 385]
price_stat_ave= get_price_stat_ave(data,decision_times)
price_stat_ave_base =get_price_stat_ave(data_base,decision_times)

list_min,list_max = get_90_quantile(price_stat_ave)
plt.figure()
plt.plot(decision_times[:-1],list_min, 'r--')
plt.plot(decision_times[:-1],list_max,'r--')
plt.plot(decision_times[:-1],price_stat_ave_base[0,:],'b')
plt.title('Sensitivity Analysis', size ='xx-large')
plt.ylabel('Price in $', size = 'x-large')
plt.xlabel('Period Index', size = 'x-large')
plt.savefig('sensitive_analysis')
#plt.xlim(decision_times)

In [ ]:
price_stat_ave_base

In [ ]:
!pip install numpy

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 20 10:26:46 2017

@author: matlabyy
"""

# -*- coding: utf-8 -*-
"""
Created on Fri Jun 16 15:16:44 2017

@author: Ted Yang
"""

import datetime as dt
from tree import TreeModel
from bau import DLWBusinessAsUsual
from cost import DLWCost
from damage_Yili import DLWDamage
from utility import EZUtility
from optimization import GeneticAlgorithm, GradientSearch
import numpy as np
import pickle
def base_case(change):
    time_list =list()
    print('Start',dt.datetime.time(dt.datetime.now()))
    time_list.append(dt.datetime.time(dt.datetime.now()))
    t = TreeModel(decision_times=[0, 15, 45, 85, 185, 285, 385])
    print('End tree',dt.datetime.time(dt.datetime.now()))
    time_list.append(dt.datetime.time(dt.datetime.now()))
    bau_default_model = DLWBusinessAsUsual()
    bau_default_model.bau_emissions_setup(tree=t)
    print('End bau',dt.datetime.time(dt.datetime.now()))
    time_list.append(dt.datetime.time(dt.datetime.now()))
    c = DLWCost(t, bau_default_model.emit_level[0], g=92.08, a=3.413, join_price=2000.0, max_price=2500.0,
					tech_const=1.5, tech_scale=0.0, cons_at_0=30460.0)
    print('End cost',dt.datetime.time(dt.datetime.now()))
    time_list.append(dt.datetime.time(dt.datetime.now()))
    df = DLWDamage(tree=t, bau=bau_default_model, cons_growth=0.015, ghg_levels=[450, 650, 1000], subinterval_len=5,change=change)    
    df.damage_simulation( draws=4, peak_temp=6.0, disaster_tail=18.0, tip_on=True, 
							 temp_map=0, temp_dist_params=None, maxh=100.0)
    print('End damage',dt.datetime.time(dt.datetime.now()))
    time_list.append(dt.datetime.time(dt.datetime.now()))
    u = EZUtility(tree=t, damage=df, cost=c, period_len=5.0, eis=0.9, ra=7.0, time_pref=0.005)
    print('End utility',dt.datetime.time(dt.datetime.now()))
    time_list.append(dt.datetime.time(dt.datetime.now()))
    ga_model = GeneticAlgorithm(pop_amount=150, num_generations=1, cx_prob=0.8, mut_prob=0.5, 
	                            bound=1.5, num_feature=63, utility=u, print_progress=True) 
    gs_model = GradientSearch(var_nums=63, utility=u, accuracy=1e-8, 
	                          iterations=1, print_progress=True)
    final_pop, fitness = ga_model.run()
    sort_pop = final_pop[np.argsort(fitness)][::-1]
    m_opt, u_opt = gs_model.run(initial_point_list=sort_pop, topk=1)

    print("SCC: ", c.price(0, m_opt[0], 0))
    print('End opt/End',dt.datetime.time(dt.datetime.now()))
    time_list.append(dt.datetime.time(dt.datetime.now()))
    result_time_list =[0]
    #change dt.time to dt.timedelta so that it can be added or minused
    temp_list =list()
    for x in time_list:
        temp_list.append( dt.timedelta(hours=x.hour, minutes=x.minute, seconds=x.second, microseconds=x.microsecond))
    for i in range(len(temp_list)-1):
        i+=1
        result_time_list.append((temp_list[i]-temp_list[i-1]).total_seconds())
    price_list = list()
    for decision_time in range(len(t.decision_times)-1):
    	start_node,end_node = t.get_nodes_in_period(decision_time)
    	average_mit = df.average_mitigation(m_opt,decision_time)
    	for index in range(end_node-start_node+1):
    		index_ori =index + start_node
    		price_list.append(c.price(t.decision_times[decision_time],m_opt[index_ori],average_mit[index]))
    return result_time_list,df.parameter_list, m_opt,u_opt,price_list
if __name__ == "__main__":
    
    result_list = list()
    for change in range(9):
        count =0
        while count <10:
	        x= base_case(change)
	        result_list.append(x)
	        count +=1
    with open('sensitive_analysis_9t30.pkl','wb') as f:
        pickle.dump(result_list,f)
#    with open('sensitive_analysis_100.pkl','rb') as inputs:
#        re = pickle.load(inputs)
#    print(re)

('Start', datetime.time(21, 50, 44, 207000))
('End tree', datetime.time(21, 50, 44, 208000))
('End bau', datetime.time(21, 50, 44, 209000))
('End cost', datetime.time(21, 50, 44, 209000))
Starting damage simulation..
('change is made', 2.81)
(0, 3.6250776095355604)
Done!
('End damage', datetime.time(21, 50, 44, 505000))
('End utility', datetime.time(21, 50, 44, 505000))
----------------Genetic Evolution Starting----------------
-- Generation 1 --
 Min nan 
 Max nan 
 Avg nan
 Std nan 
 Population Size 150
 Best Individual:  [ 1.15165103  0.62426585  0.59221197  0.08948922  1.00040553  0.44839008
  0.15570166  1.06884354  1.43004086  1.12790731  1.01364835  0.73789886
  1.19490043  1.15430111  1.15099689  0.39609317  1.25068037  0.77027636
  0.08536476  1.43304548  1.30052968  1.26223974  0.17654676  0.28310599
  1.36673987  0.40894628  0.56557295  0.12139948  0.77965602  1.38856091
  1.00141935  1.2544442   0.93514491  0.12944461  0.83455551  0.9252748
  0.72791579  0.61113161  0.95762

KeyboardInterrupt: 

In [ ]:
result_list